In [1]:
import cv2
import numpy as np
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
from mtcnn import MTCNN


In [25]:
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import load_model

In [5]:
import cv2
import numpy as np
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from mtcnn import MTCNN
import tensorflow as tf
from tensorflow.keras.models import load_model

face_recognition_model = load_model('vggface_model.h5')
mtcnn_detector = MTCNN()


labels = ['brad pitt', 'chris evans', 'tom criuse']

def recognize_face(face_crop):
    face_crop = cv2.resize(face_crop, (224, 224))
    face_crop = face_crop.astype('float32')
    face_crop /= 255.0
    face_embedding = face_recognition_model.predict(np.expand_dims(face_crop, axis=0))
    print(face_embedding)
    predicted_label_idx = np.argmax(face_embedding)
    print(face_embedding[0][predicted_label_idx])
    if face_embedding[0][predicted_label_idx] < 0.9:
        predicted_label = 'Unknown'
    else:
        predicted_label = labels[predicted_label_idx]
    
    return predicted_label

def process_image(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    faces = mtcnn_detector.detect_faces(image_rgb)
    
    for face in faces:
        x, y, w, h = face['box']
        face_crop = image[y:y+h, x:x+w]
        
        recognized_label = recognize_face(face_crop)
        
        text_size = cv2.getTextSize(recognized_label, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 2)[0]
        text_width, text_height = text_size
        text_x = max(x, 0) 
        text_y = max(y - 20, text_height)
        
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, recognized_label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
        
    img_resized = cv2.resize(image, (500, 500))
    cv2.imshow('Face Recognition Result', img_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        faces = mtcnn_detector.detect_faces(frame_rgb)
        
        for face in faces:
            x, y, w, h = face['box']
            face_crop = frame[y:y+h, x:x+w]
            
            recognized_label = recognize_face(face_crop)
            
            text_size = cv2.getTextSize(recognized_label, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 2)[0]
            text_width, text_height = text_size
            text_x = max(x, 0) 
            text_y = max(y - 20, text_height)
        
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, recognized_label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
            
        frame_resized = cv2.resize(frame, (500, 500))
        cv2.imshow('Face Recognition Video', frame_resized)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# image_path = ''
# process_image(image_path)

# Process a video
video_path = 'test_video.mp4'
process_video(video_path)


1/1 [==============================] - 1s 847ms/step
[[0.91844064 0.07738314 0.00417626]]
0.91844064
1/1 [==============================] - 0s 352ms/step
[[0.9790481  0.0156322  0.00531968]]
0.9790481
1/1 [==============================] - 0s 357ms/step
[[0.9867531  0.0094957  0.00375124]]
0.9867531
